## Note
This notebook serves as a mini tutorial on what you need to fine tune the model on a custom dataset in the simplest way.

The main things that need to be done to do so is to create a dataset class following the implementation of the Coralscapes class and to add the model checkpoint in the arguments of the training script or in the configuration file.

### Running the finetune script on top of this

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# %cd /content/drive/MyDrive/Data\ Challenge\ 3\ Group\ 3
# !git clone https://github.com/eceo-epfl/coralscapesScripts.git

In [3]:
# %cd coralscapesScripts

In [4]:
# !pip install albumentations transformers torch segmentation_models_pytorch torchmetrics peft matplotlib


# Setup

In [5]:
import sys
import os

# cur = os.path.join(os.getcwd(), '..')
# print(cur)
# proj_path = os.path.abspath(
#     os.path.join(os.getcwd(), '..')
# )
# coralpath = os.path.join(proj_path, 'coralscapesScripts')
# print(coralpath)
#
# sys.path.insert(0, coralpath)
# print(
#     sys.path
# )# # Assume your notebook is in <project_root>/coralscapes/, and coralscapesScripts is in <project_root>/
# # project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
# # if project_root not in sys.path:
# #     sys.path.insert(0, project_root)


In [6]:
# !ls
# %cd coralscapesScripts
# TODO: specify paths
root = "data_yellow_1"
root_copy = "data_yellow_1"
color = "yellow"  # these are to name the checkpoints folder
cluster = "1"


In [7]:
import torch

import albumentations as A

from torch.utils.data import DataLoader, Dataset
from coralscapesScripts.datasets.dataset import Coralscapes
from coralscapesScripts.datasets.preprocess import get_preprocessor
from coralscapesScripts.datasets.utils import calculate_weights

from coralscapesScripts.segmentation.model import Benchmark_Run
from coralscapesScripts.segmentation.benchmarking import launch_benchmark

from coralscapesScripts.visualization import show_samples

from coralscapesScripts.logger import Logger, save_benchmark_run
from coralscapesScripts.io import setup_config, get_parser, update_config_with_args
import copy

from collections import namedtuple
import os
import numpy as np
from PIL import Image
import json

#their eval sucks coz its bloated so using simpligied on
import torch
# from tqdm import tqdm
from tqdm.autonotebook import tqdm
import torch.nn as nn # Import the neural network module
import torch.nn.functional as F # Import functional for interpolation


C:\Users\20223018\.conda\envs\jbg060_4\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\20223018\.conda\envs\jbg060_4\Lib\site-packages\pydantic\_internal\_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
C:\Users\20223018\.conda\envs\jbg060_4\Lib\site-packages\pydantic\_internal\_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with v

In [8]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

In [9]:
device_count = torch.cuda.device_count()
for i in range(device_count):
    print(f"CUDA Device {i}: {torch.cuda.get_device_name(i)}")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

CUDA Device 0: NVIDIA RTX A2000 Laptop GPU


device(type='cuda')

## Config

- Currently the Benchmark_Run takes the cfg and pops out the different parts (meaning the cfg file is **changed** after the object initialization).
- If you want to look at the cfg after that, use the cfg_logger copy which will contain the original configuration.
- Keep in mind due to this that if you try to "recreate" the Benchmark_Run object without rerunning the configuration it will most likely result in an error, so better to start over (or at least just rerun the config part of the script).

# dont really get wtvr the fuck they r tryna so imma just roll w it for now
# UPDATE- u just have to rerun the whole thing coz it changes teh cfg. nothing major. ignoe

In [10]:
cfg = setup_config(config_path='configs/segformer-mit-b2_lora.yaml', config_base_path='configs/base.yaml')

## Args

### Note
- As in the command line, here the arguments take presedence over the config file, so here you can overwrite/test out some parameters without directly changing the cfg.
- Keep in mind not all parameters of the config can be changed through cmd (you can see which ones in the get_parser and update_config_with_args function).

In [11]:
args_input = "--run-name=fine_tune_notebook --batch-size=1 --batch-size-eval=1 --epochs=50"
args_input = args_input.split(" ")

parser = get_parser()
args = parser.parse_args(args_input)

cfg = update_config_with_args(cfg, args)
cfg_logger = copy.deepcopy(cfg)

# Data

### Note
Make a new dataset class using the Coralscapes class in coralscapesScripts.datasets.dataset as an example. Importantly, it needs to contain the following as arguments
- transform (albumentations)
- transform_target
- N_classes
- id2label  

In [12]:
# root="/content/drive/MyDrive/Data Challenge 3 Group 3/01_data/coral_bleaching/reef_support/UNAL_BLEACHING_TAYRONA"
# root="/content/drive/MyDrive/Data Challenge 3 Group 3/cluster_2"

id2classes = {
    # Background classes -> 0 (background)
    #ignore the label name from here, i jut saw it from the class.json we can play around w it but meh
    0: 0,   # background
    1: 0,   # seagrass
    2: 0,   # sand OR WTVR
    5: 0,   # sand or wtvr
    6: 0,   # rubble or wtvr
    7: 0,   # algae_covered_substrate
    8: 0,   # algae_covered_substrate
    9: 0,   # fish
    10: 0,  # algae_covered_substrate
    11: 0,  # algae_covered_substrate
    12: 0,  # algae_covered_substrate
    13: 0,  # background
    14: 0,  # dark
    15: 0,  # algae_covered_substrate

    # Bleached coral classes -> 1 (bleached)
    3: 1,   # other_coral_bleached
    4: 1,   # other_coral_bleached
    16: 1,  # massive_meandering_bleached -> bleached
    19: 1,  # branching_bleached -> bleached
    33: 1,  # meandering_bleached -> bleached

    # Non-bleached coral classes -> 2 (non-bleached)
    17: 2,  # massive_meandering_alive -> non-bleached
    18: 2,  # rubble (coral rubble) -> non-bleached
    20: 2,  # branching_dead -> non-bleached (dead but not bleached)
    21: 2,  # millepora -> non-bleached
    22: 2,  # branching_alive -> non-bleached
    23: 2,  # massive_meandering_dead -> non-bleached (dead but not bleached)
    24: 2,  # clam -> non-bleached
    25: 2,  # acropora_alive -> non-bleached
    26: 2,  # sea_cucumber -> non-bleached
    27: 2,  # turbinaria -> non-bleached
    28: 2,  # table_acropora_alive -> non-bleached
    29: 2,  # sponge -> non-bleached
    30: 2,  # anemone -> non-bleached
    31: 2,  # pocillopora_alive -> non-bleached
    32: 2,  # table_acropora_dead -> non-bleached (dead but not bleached)
    34: 2,  # stylophora_alive -> non-bleached
    35: 2,  # sea_urchin -> non-bleached
    36: 2,  # meandering_alive -> non-bleached
    37: 2,  # meandering_dead -> non-bleached (dead but not bleached)
    38: 2,  # crown_of_thorn -> non-bleached
    39: 2,  # dead_clam -> non-bleached
}

In [13]:
# Helper function to map 40-class predictions back to 3 classes
def map_40_to_3_classes(predictions):
    """
    Map 40-class predictions back to 3 classes:
    - 0: Background (classes 0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)
    - 1: Bleached (classes 3, 4, 16, 19, 33)
    - 2: Non-bleached (classes 17, 18, 20-32, 34-39)
    """
    # Create a mapping array
    mapping = np.zeros(40, dtype=np.uint8)

    # Set bleached classes
    bleached_classes = [3, 4, 16, 19, 33]
    for cls in bleached_classes:
        mapping[cls] = 1

    # Set non-bleached classes
    non_bleached_classes = [17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39]
    for cls in non_bleached_classes:
        mapping[cls] = 2

    # Apply mapping
    if isinstance(predictions, torch.Tensor):
        predictions = predictions.cpu().numpy()

    mapped_predictions = mapping[predictions]
    return mapped_predictions

In [14]:
# for root, dirs, files in os.walk('/content/drive/MyDrive/Data Challenge 3 Group 3/01_data/coral_bleaching/reef_support/UNAL_BLEACHING_TAYRONA'):
for root, dirs, files in os.walk(root):
    for name in dirs:
        print(os.path.join(root, name), 'idoes it exist?')


data_yellow_1\images idoes it exist?
data_yellow_1\masks_bleached idoes it exist?
data_yellow_1\masks_non_bleached idoes it exist?


In [15]:
class CoralBleachingDataset(Dataset):
    """
    Dataset for coral bleaching segmentation.
    Expects data structure:
    - images/: RGB images
    - masks_bleached/: Masks for bleached corals
    - masks_non_bleached/: Masks for non-bleached corals

    This class follows the structure of the original Coralscapes class but works with our 3-class setup.
    """
    def __init__(self, root=root, split='train', transform=None, transform_target=True):
        """
        Args:
            transform (callable): A function/transform that takes in an image and returns a transformed version.
            transform_target (bool): Whether to also transform the segmentation mask.
        """
        self.root = os.path.expanduser(root)
        self.transform = transform
        self.transform_target = transform_target
        self.split = split

        self.CoralClass = namedtuple('CoralClass', ['name', 'id', 'train_id', 'category', 'category_id', 'ignore_in_eval', 'color'])

        self.classes = []
        self.classes.append(self.CoralClass('background', 0, 0, 'background', 0, True, (0, 0, 0)))      # Black - background
        self.classes.append(self.CoralClass('bleached', 1, 1, 'bleached', 1, False, (255, 255, 255)))    # White - bleached coral
        self.classes.append(self.CoralClass('non_bleached', 2, 2, 'non_bleached', 2, False, (128, 128, 128)))  # Gray - non-bleached coral

        # Set number of classes - follow the same logic as the original Coralscapes class
        self.N_classes = 3  # We have 3 classes: background, bleached, non-bleached

        self.id2label = {0: "background", 1: "bleached", 2: "non_bleached"}
        self.label2id = {v: k for k, v in self.id2label.items()}

        self.train_id_to_color = np.array([c.color for c in self.classes])

        # juliette double check these.
        # Define mapping from original 40 classes to our 3 classes
        # This mapping is critical for interfacing with the model
        self.class_mapping = {  #Mo
            # Background classes -> 0
            0: 0, 1: 0, 2: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0,
            # Bleached coral classes -> 1
            3: 1, 4: 1, 16: 1, 19: 1, 33: 1,
            # Non-bleached coral classes -> 2
            17: 2, 18: 2, 20: 2, 21: 2, 22: 2, 23: 2, 24: 2, 25: 2, 26: 2, 27: 2, 28: 2, 29: 2,
            30: 2, 31: 2, 32: 2, 34: 2, 35: 2, 36: 2, 37: 2, 38: 2, 39: 2
        }
        # self.class_mapping = {  #mine
        #     # Background classes -> 0
        #     0: 0, 1: 0, 2: 0, 5: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 18: 2,
        #     # Bleached coral classes -> 1
        #     3: 1, 4: 1, 16: 1, 19: 1, 33: 1,
        #     # Non-bleached coral classes -> 2
        #     6: 2,
        #     17: 2, 20: 2, 21: 2, 22: 2, 23: 2, 24: 2, 25: 2, 26: 2, 27: 2, 28: 2, 29: 2,
        #     30: 2, 31: 2, 32: 2, 34: 2, 35: 2, 36: 2, 37: 2, 38: 2, 39: 2
        # }
        # Create reverse mapping for model output interpretation
        self.reverse_class_mapping = {}
        for original_class, our_class in self.class_mapping.items():
            if our_class not in self.reverse_class_mapping:
                self.reverse_class_mapping[our_class] = []
            self.reverse_class_mapping[our_class].append(original_class)

        # Create train/val/test splits
        self.images_dir = os.path.join(self.root, 'images')
        all_images = [f for f in os.listdir(self.images_dir)
                     if f.endswith(('.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG'))]

        np.random.seed(42)
        np.random.shuffle(all_images)

        if split == 'train':
            self.images = all_images[:int(0.7*len(all_images))]
        elif split == 'val':
            self.images = all_images[int(0.7*len(all_images)):int(0.85*len(all_images))]
        elif split == 'test':
            self.images = all_images[int(0.85*len(all_images)):]

        print(f"Loaded {len(self.images)} images for {split} split")

    def __getitem__(self, index):
        """
        Retrieve and transform an image and its corresponding segmentation maps.
        """
        img_name = self.images[index]
        image_path = os.path.join(self.root, 'images', img_name)

        # Get the base name without extension
        base_name = os.path.splitext(img_name)[0]

        # Load image
        image = np.array(Image.open(image_path).convert('RGB'))

        # Load bleached and non-bleached masks
        bleached_mask_path = os.path.join(self.root, 'masks_bleached', f"{base_name}_bleached.png")
        non_bleached_mask_path = os.path.join(self.root, 'masks_non_bleached', f"{base_name}_non_bleached.png")

        try:
            bleached_mask = np.array(Image.open(bleached_mask_path).convert('L'))
            non_bleached_mask = np.array(Image.open(non_bleached_mask_path).convert('L'))
        except FileNotFoundError as e:
            print(f"Error loading mask for {img_name}: {e}")
            raise

        # Create a combined mask where:
        # 0 = background, 1 = bleached, 2 = non-bleached
        target = np.zeros_like(bleached_mask, dtype=np.uint8)
        target[bleached_mask > 128] = 1      # White pixels in bleached mask -> class 1
        target[non_bleached_mask > 128] = 2  # White pixels in non-bleached mask -> class 2
        # Background remains 0 (black pixels)

        # ADD THIS: Resize both image and mask to the same size
        target_size = (512, 512)  # or whatever size you want

        # Resize image
        image_pil = Image.fromarray(image)
        image_resized = image_pil.resize(target_size)
        image = np.array(image_resized)

        # Resize mask
        target_pil = Image.fromarray(target)
        target_resized = target_pil.resize(target_size, Image.NEAREST)  # Use NEAREST for masks
        target = np.array(target_resized)

        if self.transform:
            if self.transform_target:
                transformed = self.transform(image=image, mask=target)
                image = transformed["image"]
                target = transformed["mask"]
            else:
                transformed = self.transform(image=image)
                image = transformed["image"]

        # Convert image to PyTorch format (C, H, W) if it's still a numpy array
        if isinstance(image, np.ndarray):
            image = image.transpose(2, 0, 1)

        return image, target

    def __len__(self):
        return len(self.images)

In [16]:
#resizing ffs coz images are shite.
transforms = {}
for split in cfg.augmentation:
    transform_list = [A.Resize(height=512, width=512)] # start with resize to ensure consistent size

    for transform_name, transform_params in cfg.augmentation[split].items():
        if "Crop" in transform_name and "height" in transform_params and "width" in transform_params:# skipping any crop operations that would be larger than our resize
            if transform_params["height"] > 512 or transform_params["width"] > 512:
                print(f"Warning: Adjusting '{transform_name}' to match 512x512 size in split '{split}'")
                transform_params["height"] = min(transform_params["height"], 512)
                transform_params["width"] = min(transform_params["width"], 512)

        transform_list.append(getattr(A, transform_name)(**transform_params))

    transforms[split] = A.Compose(transform_list)

In [17]:
# cfg.data.root = '/content/drive/MyDrive/Data Challenge 3 Group 3/01_data/coral_bleaching/reef_support/UNAL_BLEACHING_TAYRONA'
  # Adjust this path to your data directory FOR EACH OF THE CLUSTER/ this is v important or wasted gpu hours...
# cfg.data.root="/content/drive/MyDrive/Data Challenge 3 Group 3/cluster_2"
cfg.data.root = root_copy


In [18]:
train_dataset = CoralBleachingDataset(root=cfg.data.root, split='train', transform=transforms["train"])

transform_target = cfg.training.eval.transform_target if cfg.training.eval is not None and cfg.training.eval.transform_target is not None else True

val_dataset = CoralBleachingDataset(root=cfg.data.root, split='val', transform=transforms["val"], transform_target=transform_target)
test_dataset = CoralBleachingDataset(root=cfg.data.root, split='test', transform=transforms["test"], transform_target=transform_target)

print(f"Classes: {train_dataset.id2label}")
print(f"Number of classes: {train_dataset.N_classes}")
print(f"Train dataset has {len(train_dataset)} samples")
print(f"Validation dataset has {len(val_dataset)} samples")
print(f"Test dataset has {len(test_dataset)} samples")

Loaded 203 images for train split
Loaded 44 images for val split
Loaded 44 images for test split
Classes: {0: 'background', 1: 'bleached', 2: 'non_bleached'}
Number of classes: 3
Train dataset has 203 samples
Validation dataset has 44 samples
Test dataset has 44 samples


In [19]:
train_loader = DataLoader(train_dataset, batch_size=cfg.data.batch_size, shuffle=True, num_workers=0, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=cfg.data.batch_size_eval, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=cfg.data.batch_size_eval, shuffle=False, num_workers=0)

In [20]:
print(len(train_loader), len(val_loader), len(test_loader))

203 44 44


In [21]:
weight = calculate_weights(train_dataset).to(device) if(cfg.data.weight) else None

# Some masks folder sanity checks

## Visualizations

In [22]:
# show_samples(train_dataset, n=2)
# wont run coz i changed the class to coralscapebleachingdataset but can be visualized another way, not important yet.

In [23]:
# show_samples(val_dataset, n=2)

In [24]:
# show_samples(test_dataset, n=2)

# Benchmarking

## Model

In [25]:
benchmark_run = Benchmark_Run(run_name=cfg.run_name,
                             model_name=cfg.model.name,
                             N_classes=3,
                             device=device,
                             model_kwargs=cfg.model.kwargs,
                             model_checkpoint=cfg.model.checkpoint,
                             lora_kwargs=cfg.lora,
                             training_hyperparameters=cfg.training)

C:\Users\20223018\.conda\envs\jbg060_4\Lib\site-packages\transformers\utils\deprecation.py:172: UserWarning: The following named arguments are not valid for `SegformerImageProcessor.__init__` and were ignored: 'ignore_index', 'num_labels'
  return func(*args, **kwargs)
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b2 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight']
You shou

In [26]:
benchmark_run.print_trainable_parameters()

trainable params: 5290883 || all params: 32631494 || trainable%: 16.21


### Note
- If you want to use a LoRA approach, as long as you have it in the config (as in this case) with the correct parameters, you are good to go to the training script, no other changes need to be made.  
- If you want to freeze a specific layer, the easiest start is to just look at the model here and freeze the parameters you prefer.

In [27]:
benchmark_run.model

PeftModel(
  (base_model): LoraModel(
    (model): SegformerForSemanticSegmentation(
      (segformer): SegformerModel(
        (encoder): SegformerEncoder(
          (patch_embeddings): ModuleList(
            (0): SegformerOverlapPatchEmbeddings(
              (proj): Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
              (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
            )
            (1): SegformerOverlapPatchEmbeddings(
              (proj): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
              (layer_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
            )
            (2): SegformerOverlapPatchEmbeddings(
              (proj): Conv2d(128, 320, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
              (layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
            )
            (3): SegformerOverlapPatchEmbeddings(
              (proj): Conv2d(320, 512, kernel_si

## Logger

In [28]:
# !pip install --upgrade wandb
# import wandb


In [29]:
logger = Logger(
    project = cfg.logger.wandb_project,
    benchmark_run = benchmark_run,
    log_epochs = cfg.logger.log_epochs,
    config = cfg_logger,
    checkpoint_dir = "./"
)
# wandb api key if they ask for it, put this 9770552259c029b0835547eed1ed7173e8c8eccf

wandb: Currently logged in as: muhammadibnerafiq (muhammadibnerafiq-tilburg-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## Run

In [30]:
benchmark_run.print_trainable_parameters()

trainable params: 5290883 || all params: 32631494 || trainable%: 16.21


In [31]:
data_batch = next(iter(train_loader))
labels_batch = data_batch[1] # Access labels using index 1

print("Shape of labels batch:", labels_batch.shape)
print("Unique values in labels batch:", torch.unique(labels_batch))

Shape of labels batch: torch.Size([1, 512, 512])
Unique values in labels batch: tensor([0, 1, 2], dtype=torch.uint8)


In [32]:
# benchmark_metrics = launch_benchmark(train_loader, val_loader, test_loader, benchmark_run, logger = logger)
# save_benchmark_run(benchmark_run, benchmark_metrics)

WITHOUT K-FOLD COZ K-FOLD IS TOO MUCH...

In [33]:
def calculate_iou(pred, target, num_classes=3):
    ious = []
    pred = pred.view(-1)
    target = target.view(-1)

    for cls in range(num_classes):
        pred_inds = pred == cls
        target_inds = target == cls
        intersection = (pred_inds & target_inds).sum().float().item()
        union = (pred_inds | target_inds).sum().float().item()
        iou = intersection / union if union > 0 else 0
        ious.append(iou)

    return ious, np.mean(ious)

def train_and_validate_simplified(train_loader, val_loader, benchmark_run, criterion, logger=None, start_epoch=0, end_epoch=None, save_dir="./checkpoints", save_epochs=None):
    if end_epoch is None:
        end_epoch = benchmark_run.training_hyperparameters.epochs

    benchmark_metrics = {
        "train_loss": [],
        "val_loss": [],
        "val_accuracy": [],
        "val_miou": [],
        "val_iou_background": [],
        "val_iou_bleached": [],
        "val_iou_non_bleached": []
    }

    # Create save directory if it doesn't exist
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    for epoch in range(start_epoch, end_epoch):
        print(f"EPOCH {epoch+1}:")

        # Train
        benchmark_run.model.train() # Ensure model is in training mode

        train_loss = 0.0
        num_batches = 0
        for data in tqdm(train_loader, desc="Training"):
            # Move data to device
            inputs, labels = data
            inputs = inputs.to(benchmark_run.device)
            labels = labels.to(benchmark_run.device)

            # Zero the parameter gradients
            benchmark_run.optimizer.zero_grad()

            # Forward pass
            outputs = benchmark_run.model(inputs)

            # Resize model output (logits) to match target size
            # Assuming outputs.logits has shape (N, C, H_out, W_out) and labels have shape (N, H_target, W_target)
            # We want to resize outputs.logits to (N, C, H_target, W_target)
            resized_logits = F.interpolate(
                outputs.logits,
                size=labels.shape[-2:], # Target spatial size (H, W)
                mode='bilinear', # Use bilinear interpolation for resizing logits
                align_corners=False # Avoid alignment issues
            )

            # Calculate loss
            loss = criterion(resized_logits, labels.long())

            # Backward pass and optimize
            loss.backward()
            benchmark_run.optimizer.step()

            train_loss += loss.item()
            num_batches += 1

        if num_batches > 0:
            train_loss /= num_batches

        benchmark_metrics["train_loss"].append(train_loss)

        if hasattr(benchmark_run, "scheduler"):
            benchmark_run.scheduler.step()

        print(f'LOSS train {train_loss}')

        # Simple validation pass
        benchmark_run.model.eval()
        val_loss = 0.0
        num_batches_val = 0
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        val_ious = np.zeros(3)  # For 3 classes

        with torch.no_grad():
            for vdata in tqdm(val_loader, desc="Validating"):
                inputs, labels = vdata
                inputs = inputs.to(benchmark_run.device)
                labels = labels.to(benchmark_run.device)

                # Forward pass
                outputs = benchmark_run.model(inputs)

                # Resize model output (logits) to match target size
                resized_logits = F.interpolate(
                    outputs.logits,
                    size=labels.shape[-2:], # Target spatial size (H, W)
                    mode='bilinear', # Use bilinear interpolation for resizing logits
                    align_corners=False # Avoid alignment issues
                )

                # Calculate loss
                loss = criterion(resized_logits, labels.long())

                # Calculate accuracy
                preds = torch.argmax(resized_logits, dim=1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.numel()

                # Calculate IoU
                batch_ious, _ = calculate_iou(preds, labels)
                val_ious += np.array(batch_ious)

                val_loss += loss.item()
                num_batches_val += 1

        if num_batches_val > 0:
            val_loss /= num_batches_val
            val_ious /= num_batches_val

        val_accuracy = val_correct / val_total if val_total > 0 else 0
        val_miou = np.mean(val_ious)


        benchmark_metrics["val_loss"].append(val_loss)
        benchmark_metrics["val_accuracy"].append(val_accuracy)
        benchmark_metrics["val_miou"].append(val_miou)
        benchmark_metrics["val_iou_background"].append(val_ious[0])
        benchmark_metrics["val_iou_bleached"].append(val_ious[1])
        benchmark_metrics["val_iou_non_bleached"].append(val_ious[2])

        print(f'LOSS valid {val_loss:.4f}')
        print(f'Accuracy valid {val_accuracy:.4f}')
        print(f'mIoU valid {val_miou:.4f}')
        print(f'IoU per class: Background={val_ious[0]:.4f}, Bleached={val_ious[1]:.4f}, Non-bleached={val_ious[2]:.4f}')

        if logger and epoch % logger.log_epochs == 0:
            logger.log({
                "train/loss": train_loss,
                "val/loss": val_loss,
                "val/accuracy": val_accuracy,
                "val/miou": val_miou,
                "val/iou_background": val_ious[0],
                "val/iou_bleached": val_ious[1],
                "val/iou_non_bleached": val_ious[2]
            }, epoch)

        # Save checkpoint periodically
        if save_epochs is not None and (epoch + 1) % save_epochs == 0:
            checkpoint_path = os.path.join(save_dir, f"model_epoch_{epoch+1}.pth")
            torch.save(benchmark_run.model.state_dict(), checkpoint_path)
            print(f"Saved model checkpoint to {checkpoint_path}")


    metrics_path = os.path.join(save_dir, "benchmark_metrics.json")
    with open(metrics_path, "w") as f:
        json.dump(benchmark_metrics, f)
    return benchmark_metrics

In [34]:
# chatgpt version of k-fold should be used ideally
from sklearn.model_selection import KFold

def calculate_iou(pred, target, num_classes=3):
    ious = []
    pred = pred.view(-1)
    target = target.view(-1)

    for cls in range(num_classes):
        pred_inds = pred == cls
        target_inds = target == cls
        intersection = (pred_inds & target_inds).sum().float().item()
        union = (pred_inds | target_inds).sum().float().item()
        iou = intersection / union if union > 0 else 0
        ious.append(iou)

    return ious, np.mean(ious)

def train_and_validate_kfold(train_loader, val_loader, benchmark_run, criterion, n_splits=5, logger=None,
                            start_epoch=0, end_epoch=None, save_dir=f"./checkpoints_{color}_{cluster}", save_best_only=True):
    """
    Perform k-fold cross-validation training and validation.

    Args:
        train_loader: DataLoader for training data
        val_loader: DataLoader for validation data
        benchmark_run: Object containing model, optimizer, device, etc.
        criterion: Loss function
        n_splits: Number of folds for k-fold cross-validation
        logger: Optional logger for metrics
        start_epoch: Starting epoch number
        end_epoch: Ending epoch number
        save_dir: Directory to save model checkpoints
        save_best_only: If True, only save model when validation performance improves
    """
    if end_epoch is None:
        end_epoch = benchmark_run.training_hyperparameters.epochs

    # Create save directory if it doesn't exist
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Store metrics for each fold
    fold_metrics = []

    # Get the dataset from the train loader
    train_dataset = train_loader.dataset
    val_dataset = val_loader.dataset

    # Combine datasets for k-fold splitting
    # If they're the same dataset with different samplers, just use one
    if train_dataset == val_dataset:
        full_dataset = train_dataset
    else:
        # This assumes both datasets have the same structure and can be concatenated
        try:
            full_dataset = torch.utils.data.ConcatDataset([train_dataset, val_dataset])
        except:
            print("Warning: Could not concatenate datasets. Using training dataset for k-fold.")
            full_dataset = train_dataset

    # Initialize k-fold cross validation
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Get batch size and num_workers from original loaders
    batch_size = train_loader.batch_size
    num_workers = train_loader.num_workers

    # Track the best overall model across all folds based on the custom score
    best_overall_score = -1.0
    best_overall_fold = -1
    best_overall_epoch = -1
    best_overall_metrics = None
    best_overall_model_state_dict = None


    # Loop through each fold
    for fold, (train_idx, val_idx) in enumerate(kfold.split(range(len(full_dataset)))):
        print(f"\n{'='*20} FOLD {fold+1}/{n_splits} {'='*20}")

        # Create data loaders for this fold
        train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
        val_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)

        fold_train_loader = torch.utils.data.DataLoader(
            full_dataset,
            batch_size=batch_size,
            sampler=train_subsampler,
            num_workers=num_workers
        )

        fold_val_loader = torch.utils.data.DataLoader(
            full_dataset,
            batch_size=batch_size,
            sampler=val_subsampler,
            num_workers=num_workers
        )

        # Reset model weights for each fold if possible
        if hasattr(benchmark_run.model, 'reset_parameters'):
            benchmark_run.model.reset_parameters()
        else:
            print("Warning: Model does not have reset_parameters method. Using current weights.")

        # Reset optimizer for the new model parameters
        if hasattr(benchmark_run, 'optimizer_class') and hasattr(benchmark_run, 'optimizer_params'):
            benchmark_run.optimizer = benchmark_run.optimizer_class(
                benchmark_run.model.parameters(),
                **benchmark_run.optimizer_params
            )

        # Initialize metrics for this fold
        fold_benchmark_metrics = {
            "train_loss": [],
            "val_loss": [],
            "val_accuracy": [],
            "val_miou": [],
            "val_iou_background": [],
            "val_iou_bleached": [],
            "val_iou_non_bleached": [],
            "val_custom_score": []
        }

        # Track best performance for this fold
        best_fold_score = -1.0

        # Train for specified number of epochs
        for epoch in range(start_epoch, end_epoch):
            print(f"EPOCH {epoch+1}/{end_epoch}:")

            # Train
            benchmark_run.model.train()
            train_loss = 0.0
            num_batches = 0

            for data in tqdm(fold_train_loader, desc="Training"):
                # Move data to device
                inputs, labels = data
                inputs = inputs.to(benchmark_run.device)
                labels = labels.to(benchmark_run.device)

                # Zero the parameter gradients
                benchmark_run.optimizer.zero_grad()

                # Forward pass
                outputs = benchmark_run.model(inputs)

                # Resize model output (logits) to match target size
                resized_logits = F.interpolate(
                    outputs.logits,
                    size=labels.shape[-2:],
                    mode='bilinear',
                    align_corners=False
                )

                # Calculate loss
                loss = criterion(resized_logits, labels.long())

                # Backward pass and optimize
                loss.backward()
                benchmark_run.optimizer.step()

                train_loss += loss.item()
                num_batches += 1

            if num_batches > 0:
                train_loss /= num_batches

            fold_benchmark_metrics["train_loss"].append(train_loss)

            if hasattr(benchmark_run, "scheduler"):
                benchmark_run.scheduler.step()

            print(f'LOSS train {train_loss:.4f}')

            # Validation
            benchmark_run.model.eval()
            val_loss = 0.0
            num_batches_val = 0
            val_correct = 0
            val_total = 0
            val_ious = np.zeros(3)  # For 3 classes

            with torch.no_grad():
                for vdata in tqdm(fold_val_loader, desc="Validating"):
                    inputs, labels = vdata
                    inputs = inputs.to(benchmark_run.device)
                    labels = labels.to(benchmark_run.device)

                    # Forward pass
                    outputs = benchmark_run.model(inputs)

                    # Resize model output (logits) to match target size
                    resized_logits = F.interpolate(
                        outputs.logits,
                        size=labels.shape[-2:],
                        mode='bilinear',
                        align_corners=False
                    )

                    # Calculate loss
                    loss = criterion(resized_logits, labels.long())

                    # Calculate accuracy
                    preds = torch.argmax(resized_logits, dim=1)
                    val_correct += (preds == labels).sum().item()
                    val_total += labels.numel()

                    # Calculate IoU
                    batch_ious, _ = calculate_iou(preds, labels)
                    val_ious += np.array(batch_ious)

                    val_loss += loss.item()
                    num_batches_val += 1

            if num_batches_val > 0:
                val_loss /= num_batches_val
                val_ious /= num_batches_val

            val_accuracy = val_correct / val_total if val_total > 0 else 0
            val_miou = np.mean(val_ious)

            # Calculate custom ranking score
            # rank = alpha * accuracy + beta * mIoU
            # mIoU = 0.1 * iou_background + 0.45 * iou_bleached + 0.45 * iou_non_bleached
            alpha = 0.2
            beta = 0.8
            miou_weighted = (0.1 * val_ious[0] + 0.45 * val_ious[1] + 0.45 * val_ious[2])
            custom_score = alpha * val_accuracy + beta * miou_weighted

            # Store metrics
            fold_benchmark_metrics["val_loss"].append(val_loss)
            fold_benchmark_metrics["val_accuracy"].append(val_accuracy)
            fold_benchmark_metrics["val_miou"].append(val_miou)
            fold_benchmark_metrics["val_iou_background"].append(val_ious[0])
            fold_benchmark_metrics["val_iou_bleached"].append(val_ious[1])
            fold_benchmark_metrics["val_iou_non_bleached"].append(val_ious[2])
            fold_benchmark_metrics["val_custom_score"].append(custom_score)


            print(f'LOSS valid {val_loss:.4f}')
            print(f'Accuracy valid {val_accuracy:.4f}')
            print(f'mIoU valid {val_miou:.4f}')
            print(f'IoU per class: Background={val_ious[0]:.4f}, Bleached={val_ious[1]:.4f}, Non-bleached={val_ious[2]:.4f}')
            print(f'Custom Score valid {custom_score:.4f}')


            # Log metrics if logger is provided
            if logger and epoch % logger.log_epochs == 0:
                logger.log({
                    f"fold_{fold+1}/train/loss": train_loss,
                    f"fold_{fold+1}/val/loss": val_loss,
                    f"fold_{fold+1}/val/accuracy": val_accuracy,
                    f"fold_{fold+1}/val/miou": val_miou,
                    f"fold_{fold+1}/val/iou_background": val_ious[0],
                    f"fold_{fold+1}/val/iou_bleached": val_ious[1],
                    f"fold_{fold+1}/val/iou_non_bleached": val_ious[2],
                    f"fold_{fold+1}/val/custom_score": custom_score
                }, epoch)

            # Save best model for this fold based on custom score
            if save_best_only:
                if custom_score > best_fold_score:
                    best_fold_score = custom_score
                    checkpoint_path = os.path.join(save_dir, f"model_fold_{fold+1}_best_score.pth")
                    torch.save(benchmark_run.model.state_dict(), checkpoint_path)
                    print(f"Saved improved model checkpoint to {checkpoint_path} (Custom Score: {custom_score:.4f})")

            # Check if this is the best overall model
            if custom_score > best_overall_score:
                best_overall_score = custom_score
                best_overall_fold = fold + 1
                best_overall_epoch = epoch + 1
                best_overall_metrics = {
                    "train_loss": train_loss,
                    "val_loss": val_loss,
                    "val_accuracy": val_accuracy,
                    "val_miou": val_miou,
                    "val_iou_background": val_ious[0],
                    "val_iou_bleached": val_ious[1],
                    "val_iou_non_bleached": val_ious[2],
                    "val_custom_score": custom_score
                }
                best_overall_model_state_dict = copy.deepcopy(benchmark_run.model.state_dict())


        # Save metrics for this fold
        fold_metrics.append(fold_benchmark_metrics)

        # Save fold metrics
        metrics_path = os.path.join(save_dir, f"metrics_fold_{fold+1}.json")
        with open(metrics_path, "w") as f:
            json.dump(fold_benchmark_metrics, f)

    # Calculate and save average metrics across all folds
    avg_metrics = {
        "avg_val_miou_final": np.mean([metrics["val_miou"][-1] for metrics in fold_metrics]),
        "avg_val_accuracy_final": np.mean([metrics["val_accuracy"][-1] for metrics in fold_metrics]),
        "avg_val_iou_background_final": np.mean([metrics["val_iou_background"][-1] for metrics in fold_metrics]),
        "avg_val_iou_bleached_final": np.mean([metrics["val_iou_bleached"][-1] for metrics in fold_metrics]),
        "avg_val_iou_non_bleached_final": np.mean([metrics["val_iou_non_bleached"][-1] for metrics in fold_metrics]),
        "avg_val_custom_score_final": np.mean([metrics["val_custom_score"][-1] for metrics in fold_metrics]),
        "best_val_miou_across_folds": np.max([np.max(metrics["val_miou"]) for metrics in fold_metrics]),
        "best_val_custom_score_across_folds": np.max([np.max(metrics["val_custom_score"]) for metrics in fold_metrics]),
        "fold_metrics": fold_metrics,
        "best_overall_model": {
            "fold": best_overall_fold,
            "epoch": best_overall_epoch,
            "metrics": best_overall_metrics
        }
    }

    # Save the best overall model checkpoint
    if best_overall_model_state_dict is not None:
        best_overall_checkpoint_path = os.path.join(save_dir, "best_overall_model.pth")
        torch.save(best_overall_model_state_dict, best_overall_checkpoint_path)
        print(f"\nSaved best overall model checkpoint to {best_overall_checkpoint_path} (Custom Score: {best_overall_score:.4f})")


    # Save overall metrics
    overall_metrics_path = os.path.join(save_dir, "kfold_metrics.json")
    with open(overall_metrics_path, "w") as f:
        json.dump(avg_metrics, f)

    print(f"\nAverage Final mIoU across folds: {avg_metrics['avg_val_miou_final']:.4f}")
    print(f"Best mIoU across all folds: {avg_metrics['best_val_miou_across_folds']:.4f}")
    print(f"Average Final Custom Score across folds: {avg_metrics['avg_val_custom_score_final']:.4f}")
    print(f"Best Custom Score across all folds: {avg_metrics['best_val_custom_score_across_folds']:.4f}")
    print(f"\nBest overall model found in Fold {best_overall_fold}, Epoch {best_overall_epoch} with Custom Score: {best_overall_score:.4f}")
    print("="*50)

    return avg_metrics

In [35]:
%%time
criterion = nn.CrossEntropyLoss(weight=weight) # Use the calculated weight
save_directory = "./custom_checkpoints"
save_frequency_epochs = 1 # Set to an integer to save checkpoints

# benchmark_metrics = train_and_validate_simplified(
#     train_loader,
#     val_loader,
#     benchmark_run,
#     criterion=criterion,
#     logger=logger,
#     save_dir=save_directory,
#     save_epochs=save_frequency_epochs # Pass the save frequency
# )
benchmark_metrics = train_and_validate_kfold(
    train_loader,
    val_loader,
    benchmark_run,
    criterion,
    n_splits=5,
    save_best_only=True
)


==================== FOLD 1/5 ====================
EPOCH 1/50:


Training: 100%|██████████| 197/197 [01:28<00:00,  2.22it/s]


LOSS train 0.7815


Validating: 100%|██████████| 50/50 [00:20<00:00,  2.39it/s]


LOSS valid 0.5588
Accuracy valid 0.7599
mIoU valid 0.4921
IoU per class: Background=0.6940, Bleached=0.4108, Non-bleached=0.3715
Custom Score valid 0.4892
Saved improved model checkpoint to ./checkpoints_yellow_0\model_fold_1_best_score.pth (Custom Score: 0.4892)
EPOCH 2/50:


...

LOSS valid 1.0092
Accuracy valid 0.7972
mIoU valid 0.6133
IoU per class: Background=0.7504, Bleached=0.5669, Non-bleached=0.5225
Custom Score valid 0.6116
EPOCH 50/50:


Training: 100%|██████████| 198/198 [01:27<00:00,  2.27it/s]


LOSS train 0.1305


Validating: 100%|██████████| 49/49 [00:21<00:00,  2.31it/s]


LOSS valid 1.0281
Accuracy valid 0.7740
mIoU valid 0.5811
IoU per class: Background=0.7388, Bleached=0.5403, Non-bleached=0.4640
Custom Score valid 0.5755

Saved best overall model checkpoint to ./checkpoints_yellow_0\best_overall_model.pth (Custom Score: 0.6694)

Average Final mIoU across folds: 0.5334
Best mIoU across all folds: 0.6682
Average Final Custom Score across folds: 0.5288
Best Custom Score across all folds: 0.6694

Best overall model found in Fold 5, Epoch 7 with Custom Score: 0.6694
CPU times: total: 1d 59min 57s
Wall time: 8h 22min 33s
